In [5]:
import torch
import torch.nn as nn

In [6]:
import sympy as sp
from sympy import Matrix,MatrixSymbol
from sympy import symbols, diff, init_printing

init_printing()

# Jacobian matrix

假设存在函数$f: \mathbb{R}^{n} \rightarrow \mathbb{R}^{m}$, 输入变量$\mathbf{x} \in \mathbb{R}^{n}$, 输出变量为$\mathbf{f}(\mathbf{x}) \in \mathbb{R}^{m}$, 函数$f$的`Jacobian`矩阵为$m \times n$矩阵, 定义为:

$$\mathbf{J}=\left[ \begin{array}{ccc}{\frac{\partial \mathbf{f}}{\partial x_{1}}} & {\cdots} & {\frac{\partial \mathbf{f}}{\partial x_{n}}}\end{array}\right]=\left[ \begin{array}{ccc}{\frac{\partial f_{1}}{\partial x_{1}}} & {\cdots} & {\frac{\partial f_{1}}{\partial x_{n}}} \\ {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial f_{m}}{\partial x_{1}}} & {\cdots} & {\frac{\partial f_{m}}{\partial x_{n}}}\end{array}\right]$$

或者有,

$$
\mathbf{J}_{i j}=\frac{\partial f_{i}}{\partial x_{j}}
$$


## 实例1
下面看一个例子,

假设:

$$
\begin{aligned} x & \rightarrow y \rightarrow z \\ y=& f(x), z=g(y) \\ f& : \mathbb{R}^{a}  \rightarrow \mathbb{R}^{b} \\ g & : \mathbb{R}^{b} \rightarrow \mathbb{R}^{c} \end{aligned}
$$

则有:

$$
\mathbf{J}_{\mathbf{x} \rightarrow \mathbf{y}}=\left[ \begin{array}{ccc}{\frac{\partial f_{1}}{\partial x_{1}}} & {\cdots} & {\frac{\partial f_{1}}{\partial x_{a}}} \\ {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial f_{b}}{\partial x_{1}}} & {\cdots} & {\frac{\partial f_{b}}{\partial x_{a}}}\end{array}\right], \mathbf{J}_{\mathbf{y} \rightarrow \mathbf{z}}=\left[ \begin{array}{ccc}{\frac{\partial g_{1}}{\partial y_{1}}} & {\cdots} & {\frac{\partial g_{1}}{\partial y_{b}}} \\ {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial g_{c}}{\partial y_{1}}} & {\cdots} & {\frac{\partial g_{c}}{\partial y_{b}}}\end{array}\right]
$$

令:

$$
\mathbf{x}=\left[ \begin{array}{ll}{x_{1},} & {x_{2}}\end{array}\right]=\left[ \begin{array}{ll}{1,} & {2}\end{array}\right], \mathbf{y}=\left[ \begin{array}{cc}{2 x_{1}+x_{2}^{2},} & {x_{1}^{2}+2 x_{2}^{3}}\end{array}\right]
$$

所以:

$$
\mathbf{J}_{\mathbf{x} \rightarrow \mathbf{y}}=\left[ \begin{array}{cc}{2,} & {2 x_{2}} \\ {2 x_{1},} & {6 x_{2}^{2}}\end{array}\right]=\left[ \begin{array}{cc}{2,} & {4} \\ {2,} & {24}\end{array}\right]
$$

`PyTorch`不能直接计算`Jacobian`矩阵, 但是可以计算:

$$
\mathbf{J}^{T} \cdot v=\left( \begin{array}{ccc}{\frac{\partial y_{1}}{\partial x_{1}}} & {\cdots} & {\frac{\partial y_{m}}{\partial x_{1}}} \\ {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial y_{1}}{\partial x_{n}}} & {\cdots} & {\frac{\partial y_{m}}{\partial x_{n}}}\end{array}\right) \left( \begin{array}{c}{\frac{\partial l}{\partial y_{1}}} \\ {\vdots} \\ {\frac{\partial l}{\partial y_{m}}}\end{array}\right)=\left( \begin{array}{c}{\frac{\partial l}{\partial x_{1}}} \\ {\vdots} \\ {\frac{\partial l}{\partial x_{n}}}\end{array}\right) = v^T \cdot \mathbf{J}
$$

因此下面的代码:

```python
y.backward(torch.tensor([[k1, k2]]))
```

结果为:

$$
\left[ \begin{array}{ll}{k_{1},} & {k_{2}}\end{array}\right]
\left[ \begin{array}{cc}{2,} & {2 x_{2}} \\ {2 x_{1},} & {6 x_{2}^{2}}\end{array}\right]
=
k_{1} *\left[2,2 x_{2}\right]+k 2 *\left[2 x_{1}, 6 x_{2}^{2}\right]
$$

### 用Sympy推导

In [7]:
X = Matrix(symbols('x:2'))
Y = sp.zeros(2, 1)
Y[0] = 2*X[0] + X[1]**2
Y[1] = X[0]**2 + 2*X[1]**3

In [8]:
Y

⎡         2 ⎤
⎢2⋅x₀ + x₁  ⎥
⎢           ⎥
⎢  2       3⎥
⎣x₀  + 2⋅x₁ ⎦

In [9]:
J_Y = Y.jacobian(X)
J_Y

⎡ 2    2⋅x₁ ⎤
⎢           ⎥
⎢          2⎥
⎣2⋅x₀  6⋅x₁ ⎦

In [10]:
V = Matrix(symbols('v:2'))

In [11]:
OUT = V.T * J_Y

In [12]:
OUT.subs({X[0]:1, X[1]:2, V[0]:1, V[1]:1})

[4  28]

In [13]:
OUT.subs({X[0]:1, X[1]:2, V[0]:2, V[1]:1})

[6  32]

### 用torch验证

In [14]:
x = torch.tensor([[1.0, 2.0]], requires_grad=True)
y = torch.zeros_like(x)

x1 = x[0, 0]
x2 = x[0, 1]
y[0, 0] = 2*x1+x2**2
y[0, 1] = x1**2+2*x2**3

In [15]:
x, y

(tensor([[1., 2.]], requires_grad=True),
 tensor([[ 6., 17.]], grad_fn=<CopySlices>))

In [16]:
if x.grad:
    x.grad.zero_()
y.backward(torch.tensor([[1.0, 0.0]]), retain_graph=True)

x.grad

tensor([[2., 4.]])

In [17]:
if x.grad is not None:
    x.grad.zero_()
y.backward(torch.tensor([[0.0, 1.0]]), retain_graph=True)

x.grad

tensor([[ 2., 24.]])

In [18]:
if x.grad is not None:
    x.grad.zero_()
y.backward(torch.tensor([[1.0, 2.0]]), retain_graph=True)

x.grad

tensor([[ 6., 52.]])

In [19]:
if x.grad is not None:
    x.grad.zero_()
y.backward(torch.tensor([[2.0, 1.0]]), retain_graph=True)

x.grad

tensor([[ 6., 32.]])

In [20]:
if x.grad is not None:
    x.grad.zero_()
y.backward(torch.tensor([[1.0, 1.0]]), retain_graph=True)

x.grad

tensor([[ 4., 28.]])

## 实例2

假设有: 

$$
\mathbf{c}^{m \times n}=\mathbf{a}^{m \times k} \times \mathbf{b}^{k \times n},
$$

即:

$$
\left[ \begin{array}{ccc}{c_{1,1}} & {\cdots} & {c_{1, n}} \\ {\vdots} & {\ddots} & {\vdots} \\ {c_{m, 1}} & {\cdots} & {c_{m, n}}\end{array}\right]=\left[ \begin{array}{ccc}{a_{1,1}} & {\cdots} & {a_{1, k}} \\ {\vdots} & {\ddots} & {\vdots} \\ {a_{m, 1}} & {\cdots} & {a_{m, k}}\end{array}\right] \times \left[ \begin{array}{ccc}{b_{1,1}} & {\cdots} & {b_{1, n}} \\ {\vdots} & {\ddots} & {\vdots} \\ {b_{k, 1}} & {\cdots} & {b_{k, n}}\end{array}\right]
$$

`PyTorch`自动求导过程如下:

$$
\frac{\partial c_{1,1}}{\partial \mathbf{b}^{k \times n}}=\left[ \begin{array}{ccc}{\frac{\partial c_{1,1}}{\partial b_{1,1}}} & {\cdots} & {\frac{\partial c_{1,1}}{\partial b_{1, n}}} \\ {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial c_{1,1}}{\partial b_{k, 1}}} & {\cdots} & {\frac{\partial c_{1,1}}{\partial b_{k, n}}}\end{array}\right]
$$

$$
\nabla_{\mathbf{b}^{k \times n} \mathbf{c}}=\sum_{i=1}^{m} \sum_{j=1}^{n} \frac{\partial c_{i, j}}{\partial \mathbf{b}^{k \times n}}
$$

下面手动计算一下, 令:

$$
\mathbf{a}=\left[ \begin{array}{ll}{a_{1,1}} & {a_{1,2}} \\ {a_{2,1}} & {a_{2,2}}\end{array}\right]=\left[ \begin{array}{ll}{1} & {2} \\ {3} & {4}\end{array}\right],
$$

$$
\mathbf{b}=\left[ \begin{array}{lll}{b_{1,1}} & {b_{1,2}} & {b_{1,3}} \\ {b_{2,1}} & {b_{2,2}} & {b_{2,3}}\end{array}\right]=\left[ \begin{array}{lll}{1} & {2} & {3} \\ {4} & {5} & {6}\end{array}\right],
$$

$$
\mathbf{c}=\left[ \begin{array}{lll}{c_{1,1}} & {c_{1,2}} & {c_{1,3}} \\ {c_{2,1}} & {c_{2,2}} & {c_{2,3}}\end{array}\right],
$$

根据乘法公式:

$$
\begin{aligned} c_{1,1} &=a_{1,1} b_{1,1}+a_{1,2} b_{2,1} \\ c_{1,2} &=a_{1,1} b_{1,2}+a_{1,2} b_{2,2} \\ c_{1,3} &=a_{1,1} b_{1,3}+a_{1,2} b_{2,3} \\ c_{2,1} &=a_{2,1} b_{1,1}+a_{2,2} b_{2,2} \\ c_{2,2} &=a_{2,1} b_{1,2}+a_{2,2} b_{2,2} \\ c_{2,3} &=a_{2,1} b_{1,3}+a_{2,2} b_{2,2} \end{aligned}
$$

因此导数为:

$$
\frac{\partial c_{1,1}}{\partial \mathbf{b}}=\left[ \begin{array}{lll}{a_{1,1}} & {0} & {0} \\ {a_{1,2}} & {0} & {0}\end{array}\right]
$$

$$
\frac{\partial c_{1,2}}{\partial \mathbf{b}}=\left[ \begin{array}{lll}{0} & {a_{1,1}} & {0} \\ {0} & {a_{1,2}} & {0}\end{array}\right]
$$

$$
\frac{\partial c_{1,3}}{\partial \mathbf{b}}=\left[ \begin{array}{lll}{0} & {0} & {a_{1,1}} \\ {0} & {0} & {a_{1,2}}\end{array}\right]
$$

$$
\frac{\partial c_{2,1}}{\partial \mathbf{b}}=\left[ \begin{array}{lll}{a_{2,1}} & {0} & {0} \\ {a_{2,2}} & {0} & {0}\end{array}\right]
$$

$$
\frac{\partial c_{2,2}}{\partial \mathbf{b}}=\left[ \begin{array}{lll}{0} & {a_{2,1}} & {0} \\ {0} & {a_{2,2}} & {0}\end{array}\right]
$$

$$
\frac{\partial c_{2,3}}{\partial \mathbf{b}}=\left[ \begin{array}{lll}{0} & {0} & {a_{2,1}} \\ {0} & {0} & {a_{2,2}}\end{array}\right]
$$

将上面各式加起来可得:

$$
\nabla_{\mathbf{b}^{2 \times 3} \mathbf{c}}=\sum_{i=1}^{2} \sum_{j=1}^{3} \frac{\partial c_{i, j}}{\partial \mathbf{b}^{k \times n}}
=
\left[ \begin{array}{ccc}{a_{1,1}+a_{2,1}} & {a_{1,1}+a_{2,1}} & {a_{1,1}+a_{2,1}} \\ {a_{1,2}+a_{2,2}} & {a_{1,2}+a_{2,2}} & {a_{1,2}+a_{2,2}}\end{array}\right]
$$

上式就是`b.grad`的结果, 其中`c.backward(torch.ons_like(c))`中`backward()`的参数是与$\mathbf{c}^{m \times n}$ 形状相同且全为1的矩阵, 其中矩阵每个位置的值**对应**上面各式的系数. 将`c.backward(v)`的参数记为$\mathbf{v}^{m \times n}$:

$$
\text{b.grad} =  
\sum_{i=1}^{m} \sum_{j=1}^{n} v_{i, j} \times \frac{\partial c_{i, j}}{\partial \mathbf{b}}
$$

### 用Sympy推导上面的公式

In [22]:
def diff_matrix(m0, m1):
    """
    Return d_m0/d_m1.
    
    Args:
        m0, m1 are sympy.Matrix
    """
    shape = m0.shape
    out = Matrix.zeros(*m1.shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            out += diff(m0[i, j], m1)
    return out

In [38]:
A = Matrix(MatrixSymbol('a', 2, 2))
B = Matrix(MatrixSymbol('b', 2, 3))

In [39]:
C = A@B

In [40]:
dC_dA = diff_matrix(C, A)
dC_dA

⎡b₀₀ + b₀₁ + b₀₂  b₁₀ + b₁₁ + b₁₂⎤
⎢                                ⎥
⎣b₀₀ + b₀₁ + b₀₂  b₁₀ + b₁₁ + b₁₂⎦

In [53]:
dC_dA.subs({k:v for k, v in zip(B, range(1, 7))})

⎡6  15⎤
⎢     ⎥
⎣6  15⎦

In [41]:
dC_dB = diff_matrix(C, B)
dC_dB

⎡a₀₀ + a₁₀  a₀₀ + a₁₀  a₀₀ + a₁₀⎤
⎢                               ⎥
⎣a₀₁ + a₁₁  a₀₁ + a₁₁  a₀₁ + a₁₁⎦

In [52]:
dC_dB.subs({k:v for k,v in zip(A,range(1, 5))})

⎡4  4  4⎤
⎢       ⎥
⎣6  6  6⎦

### 在Pytorch中验证

In [9]:
a = torch.tensor([[1, 2], [3, 4]], 
                 dtype=torch.float32, requires_grad=True)
b = torch.tensor([[1, 2, 3], [4, 5, 6]], 
                 dtype=torch.float32, requires_grad=True)
c = torch.mm(a, b)
v = torch.ones_like(c)
c.backward(v, retain_graph=True)

In [10]:
a.grad

tensor([[ 6., 15.],
        [ 6., 15.]])

In [11]:
b.grad

tensor([[4., 4., 4.],
        [6., 6., 6.]])